In [2]:
from typing import Optional, Callable, List, Dict, Tuple
import os

import dgmc
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import torch
from torch import Tensor
import funcs
import pytorch_lightning as pl
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric


%reload_ext autoreload
%autoreload 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
file_path = f"s3://drose-sandbox/sizmek_zync_1m"
df_raw = pd.read_parquet(file_path)
print(f"Shape: {df_raw.shape[0]:,} Memory: {df_raw.memory_usage(deep=True).sum()/1e9:.2f}GB")

Shape: 1,000,000 Memory: 2.72GB


In [4]:
sizmek_cols = ["account_id", "url", "city_code",
               "state_code", "dma_code", "country_code"]
zync_cols = ["session_id", "referrer", "client", "user_agent_platform",
             "user_agent_language", "user_agent_browser"]

df = df_raw[sizmek_cols + zync_cols]
print(df.shape)
df = df[df[sizmek_cols + zync_cols].astype(bool)]  
print(df.shape)
df = df.dropna()
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df = df.reset_index(drop=True)

group_list = [e for e in (sizmek_cols + zync_cols) if e not in ("url", "referrer")]
df = df.groupby(group_list)[["url", "referrer"]].agg(lambda x: list(x))
df = df.reset_index()

sizmek_df = df[sizmek_cols]
zync_df = df[zync_cols]

(1000000, 12)
(1000000, 12)
(67894, 12)
(8386, 12)


In [5]:
sizmek_df.head(3)

,account_id,url,city_code,state_code,dma_code,country_code
0,19967,"[milesquest.com, theroot.com, https://milesque...",4049979,AL,630,US
1,19967,"[caninehq.com, https://caninehq.com/can-dogs-e...",4049979,AL,630,US
2,19967,"[pagesix.com, cbssports.com, https://pagesix.c...",4049979,AL,630,US


In [6]:
zync_df.head(3)

,session_id,referrer,client,user_agent_platform,user_agent_language,user_agent_browser
0,1ca15a3a-fc41-466b-8ecc-2ad5f0997f80:162840229...,[https://d7b2b2ad4fc14f6b19acf619d54f1e63.safe...,sizmek,android,11,chrome
1,1cc3d46f-ea7e-400b-bf16-6462dbdf69de:1628096090.3,[https://www.yahoo.com/lifestyle/14-9-million-...,sizmek,android,11,chrome
2,6f41bed7-a384-4540-bcb9-65ed800e62e6:162524377...,"[https://www.wyndhamhotels.com/, https://www.w...",sizmek,android,11,chrome


# Make new edges

In [16]:
edge_list = funcs.connect_edges_2(sizmek_df, "url")
edge_list = edge_list[edge_list['source'] != edge_list['target']]
edge_list

,source,target,column,value
1,0,398,url,milesquest.com
5,0,65,url,oo-syringe.com
7,0,88,url,newsweek.com
8,0,445,url,newsweek.com
12,1,4,url,yahoo.com
...,...,...,...,...
101164,1106,860,url,https://variety.com/2021/tv/news/mike-richards...
101165,1106,885,url,https://variety.com/2021/tv/news/mike-richards...
101166,1106,1029,url,https://variety.com/2021/tv/news/mike-richards...
101167,1106,1104,url,https://variety.com/2021/tv/news/mike-richards...


In [24]:
class ZetaDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root: str, sizmek_df: pd.DataFrame, zync_df: pd.DataFrame, column: str, 
                 label: str, feature_cols=None, parse_url=False, expand_x=None, transform=None, 
                 pre_transform=None
        ):
        self.root = root
        self.sizmek_df = sizmek_df
        self.zync_df = zync_df
        self.column = column
        self.label = label
        self.feature_cols = feature_cols
        self.parse_url = parse_url
        self.expand_x = expand_x
        super(ZetaDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        print("Removing processed file. . .")
        os.remove(self.processed_paths[0])

    @property
    def raw_file_names(self) -> List[str]:
        return [
            "sizmek_bidstream_raw_20210625_10k.csv", 
            "zync_session_tracking_orc_20210625_10k.csv"
        ]

    @property
    def processed_file_names(self):
        return ["ZetaDataset.pt"]
    
    @property
    def edge_count_1(self):
        return zeta_data.data.edge_index1.shape[1]
    
    @property
    def edge_count_2(self):
        return zeta_data.data.edge_index2.shape[1]

    def download(self):
        pass

    def process(self):
        x1, edge_index1 = self.process_graph(self.sizmek_df, self.column[0], self.feature_cols["sizmek"])
        x2, edge_index2 = self.process_graph(self.zync_df, self.column[1], self.feature_cols["zync"], 715)

        train_y = self.process_y()
        test_y = self.process_y()

        data = Data(x1=x1, edge_index1=edge_index1, x2=x2,
                    edge_index2=edge_index2, train_y=train_y,
                    test_y=test_y)
        torch.save(self.collate([data]), self.processed_paths[0])

    def process_graph(self, df, column: str, feature_cols: List, expand_x: int=None):
        print(f"Processing graph on {column}")
        #df = pd.read_csv(file_path, low_memory=False)
        #df.columns = [i.split(".")[1] for i in df.columns]

        # parse URLs
        if self.parse_url == True and column in ["url", "referrer"]:
            df[column] = df[column].apply(
                lambda x:funcs.urlparse(x).netloc if pd.notnull(x) else x
            )

        # Encode features
        feature_enc = OneHotEncoder(handle_unknown="ignore")
        features = pd.DataFrame(
            feature_enc.fit_transform(df[feature_cols]).toarray(), 
            columns=feature_enc.get_feature_names(feature_cols)
        )
        if expand_x is not None:
            print(f"Expanding X to {expand_x}")
            new_cols = [f"fake_{expand_x-i}" for i in range(expand_x - features.shape[1])][::-1]
            for col in new_cols:
                features[col] = 0
        x = torch.tensor(features.values, dtype=torch.float)
        
        #edges = funcs.connect_edges(df, column)
        #edge_index = torch.tensor(
        #    edges[['source','target']].T.values, dtype=torch.long
        #)
        
        edges = funcs.connect_edges_2(df, column)
        edge_index = torch.tensor(
            edges[["source", "target"]].T.values, dtype=torch.long
        )

        return x, edge_index

    def process_y(self) -> Tensor:
        y_1 = torch.tensor([range(0,1_000)])[0]
        y_2 = torch.tensor([range(0,1_000)])[0]
        train_y = torch.stack([y_1, y_2], dim=0)
        return train_y
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.edge_count_1:,}/{self.edge_count_2:,} edges)"

In [25]:
class SumEmbedding(object):
    def __call__(self, data):
        data.x1, data.x2 = data.x1.sum(dim=1), data.x2.sum(dim=1)
        return data

In [26]:
#sizmek_small = sizmek_df.sample(n=20_000, random_state=0)
#zync_small = zync_df.iloc[sizmek_small.index,:]

sizmek_small = sizmek_df.copy()
zync_small = zync_df.copy()

In [27]:
feature_cols = {
    "sizmek": ["account_id", "city_code","state_code", "dma_code", "country_code"],
    "zync": ["client", "user_agent_platform","user_agent_language", "user_agent_browser"]
}

zeta_data = ZetaDataset(
    root="./data/",
    sizmek_df = sizmek_small,
    zync_df = zync_small,
    column=["url", "referrer"],
    label=["zeta_user_id", "client_id"],
    feature_cols=feature_cols,
    parse_url=False,
    transform=SumEmbedding()
)
zeta_data

Processing...
Processing graph on url
Processing graph on referrer
Expanding X to 715


/tmp/ipykernel_23610/3005345540.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col] = 0


Done!
Removing processed file. . .


ZetaDataset(101,208/187,554 edges)

In [28]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--dim', type=int, default=256)
parser.add_argument('--rnd_dim', type=int, default=32)
parser.add_argument('--num_layers', type=int, default=3)
parser.add_argument('--num_steps', type=int, default=10)
parser.add_argument('--k', type=int, default=10)
args = parser.parse_args("")


psi_1 = dgmc.models.RelCNN(zeta_data.data.x1.size(-1), args.dim, args.num_layers, batch_norm=False,
               cat=True, lin=True, dropout=0.5)
psi_2 = dgmc.models.RelCNN(args.rnd_dim, args.rnd_dim, args.num_layers, batch_norm=False,
               cat=True, lin=True, dropout=0.0)

psi_1
psi_2

RelCNN(32, 32, num_layers=3, batch_norm=False, cat=True, lin=True, dropout=0.0)

In [29]:
model = dgmc.models.DGMC(psi_1, psi_2, num_steps=None, k=args.k).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
data = zeta_data.data

def train():
    model.train()
    optimizer.zero_grad()

    _, S_L = model(data.x1, data.edge_index1, None, None, data.x2,
                   data.edge_index2, None, None, data.train_y)

    loss = model.loss(S_L, data.train_y)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test():
    model.eval()

    _, S_L = model(data.x1, data.edge_index1, None, None, data.x2,
                   data.edge_index2, None, None)

    hits1 = model.acc(S_L, data.test_y)
    hits10 = model.hits_at_k(10, S_L, data.test_y)

    return hits1, hits10

In [ ]:
model.num_steps = 0
with torch.profiler.profile(
    schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=6,
        repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler("./logs/gnn_v5"),
    with_stack=True
) as profiler:
    for epoch in range(1, 500):
        print("Epoch: {}".format(epoch))
        loss = train()
        profiler.step()
        
        if epoch % 5 == 0:
            hits1, hits10 = test()
            print((f"{epoch:03d}: Loss: {loss:.4f}, Hits@1: {hits1:.4f}, "
               f"Hits@10: {hits10:.4f}"))

Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
005: Loss: 3.0029, Hits@1: 0.0060, Hits@10: 0.0270
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
010: Loss: 2.9946, Hits@1: 0.0080, Hits@10: 0.0370
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
015: Loss: 2.9889, Hits@1: 0.0050, Hits@10: 0.0420
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
020: Loss: 2.9803, Hits@1: 0.0040, Hits@10: 0.0400
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
025: Loss: 2.9652, Hits@1: 0.0030, Hits@10: 0.0380
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
030: Loss: 2.9380, Hits@1: 0.0020, Hits@10: 0.0350
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
035: Loss: 2.9101, Hits@1: 0.0040, Hits@10: 0.0410
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
040: Loss: 2.8735, Hits@1: 0.0080, Hits@10: 0.0480
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
045: Loss: 2.8401, Hits@1: 0.0080, Hits@10: 0.0460
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
050: Loss: 2.8246, Hits@1: 0.0100, Hits@10: 0.0560

In [ ]:
print('Optimize initial feature matching...')
model.num_steps = 0
for epoch in range(1, 6):
    if epoch == 3:
        print('Refine correspondence matrix...')
        model.num_steps = args.num_steps
        model.detach = True

    loss = train()

    #if epoch % 10 == 0 or epoch > 100:
    if True:
        hits1, hits10 = test()
        print((f'{epoch:03d}: Loss: {loss:.4f}, Hits@1: {hits1:.4f}, '
               f'Hits@10: {hits10:.4f}'))